In [2]:
training_directory="D:/MLDatasets/TCGA-GBM Datasets/Training"

In [3]:
from glob import glob
import os
flair_files = sorted(glob(os.path.join(training_directory, f"images_{0}", "*.nii.gz")))
t1_files = sorted(glob(os.path.join(training_directory, f"images_{1}", "*.nii.gz")))
t1Gd_files = sorted(glob(os.path.join(training_directory, f"images_{2}", "*.nii.gz")))
t2_files = sorted(glob(os.path.join(training_directory, f"images_{3}", "*.nii.gz")))
mask_files = sorted(glob(os.path.join(training_directory, f"segmentation", "*.nii.gz")))
print(len(flair_files))
print(len(t1_files))
print(len(t1Gd_files))
print(len(t2_files))
print(len(mask_files))

102
102
102
102
102


In [12]:
import torch
from tqdm.notebook import tqdm
import nibabel as nib
import numpy as np
from monai.transforms import (
    Resize,
    Compose
)

current_using = False
spatial_size=[128, 128, 144]
smaller_transform = Compose(
    Resize([128, 128, 144])
)

for i in tqdm(range(len(flair_files))):
    flair_image = nib.load(flair_files[i]).get_fdata()
    t1_image = nib.load(t1_files[i]).get_fdata()
    t1Gd_image = nib.load(t1Gd_files[i]).get_fdata()
    t2_image = nib.load(t2_files[i]).get_fdata()
    combined_image = np.stack([flair_image, t1_image, t1Gd_image, t2_image], axis=3)

    mask = nib.load(mask_files[i]).get_fdata()
    mask = mask.astype(np.uint8)

    flair_image = smaller_transform(flair_image)
    t1_image = smaller_transform(t1_image)
    t1Gd_image = smaller_transform(t1Gd_image)
    t2_image = smaller_transform(t2_image)
    
    if(i == 0):
        print(f"Combined Image shape: {combined_image.shape}")
        print(f"Mask shape: {mask.shape}")
    
    combined_image = nib.Nifti1Image(combined_image, affine=np.eye(4))
    mask = nib.Nifti1Image(mask, affine=np.eye(4))

    if current_using == True:
        nib.save(combined_image, os.path.join(training_directory, "imagesv5", f"image_{i}.nii.gz"))
        nib.save(mask, os.path.join(training_directory, "segmentationv5", f"mask_{i}.nii.gz"))
    

  0%|          | 0/102 [00:00<?, ?it/s]

Combined Image shape: (240, 240, 155, 4)
Mask shape: (240, 240, 155)
